In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/players/"

In [ ]:
dfa = []
for year in range(2013,2023):
    df1 = pd.read_parquet(box_DIR+f"NBA_Box_P_Base_{year}_PS.parquet")
    df1["GAME_ID"] = df1["GAME_ID"].astype(int)
    df2 = pd.read_parquet(box_DIR+f"NBA_Box_P_Adv_{year}_PS.parquet",columns=["gameId","personId","possessions"])
    df2.columns = ['GAME_ID','PLAYER_ID',"POSS"]
    df3 = pd.merge(df1,df2,on=['GAME_ID','PLAYER_ID'])
    df3["Season"] = year +1
    df3["DRE"] = -8.42364*df3["POSS"]/100+0.79231*df3["PTS"]-0.71944*(df3["FGA"]-df3["FG3A"])-0.55233*df3["FG3A"]-0.15944*df3["FTA"]+0.13479*df3["OREB"]+0.3996*df3["DREB"]+0.54415*df3["AST"]+1.68007*df3["STL"]+0.76387*df3["BLK"]-1.3599*df3["TOV"]-0.10838*df3["PF"]

    df3["GmSc"] = df3["PTS"]+0.4*df3["FGM"]-0.7*df3["FGA"]-0.4*(df3["FTA"]-df3["FTM"])+0.7*df3["OREB"]+0.3*df3["DREB"]+df3["STL"]+0.7*df3["AST"]+0.7*df3["BLK"]-0.4*df3["PF"]-df3["TOV"]
    dfa.append(df3)
df3 = pd.concat(dfa)

In [ ]:
players = ["LeBron James","Stephen Curry","Kevin Durant","Kawhi Leonard","DeMar DeRozan","Giannis Antetokounmpo","Luka Doncic","Shai Gilgeous-Alexander","Nikola Jokic","Jayson Tatum"]
players = ["James Harden"]
# players = ["Joel Embiid"]
pIDs = [get_pID(pp) for pp in players]
df4 = df3[df3["PLAYER_ID"].isin(pIDs)]

In [ ]:
df4g = df4.groupby(["PLAYER_NAME","Season"])
keys = list(df4g.groups)
dfb = []
for key in keys:
    df5 = df4g.get_group(key).reset_index(drop=True).reset_index()
    df5["index"] +=1
    df5 = df5.rename(columns={"index":"Game_No"})
    dfb.append(df5)
df6 = pd.concat(dfb)
df6["Player_Season"] = df6["Season"].astype(str) + " " + df6["PLAYER_NAME"]
df6["Season"] = df6["Season"].astype("category")

In [ ]:
p = (
    ggplot(df6,aes(x="Game_No",y="DRE",color="Season",group="Season"))
    + geom_point(size=2)
    + geom_smooth(se=False,size=2,method="lowess")#method="lm")
    + theme_idv
    + geom_hline(yintercept=0,linetype="dashed")
    # + scale_x_date(date_labels = "%b-%d",date_breaks = "1 month")
    # + scale_color_brewer(type="qual", palette=3)
    # + scale_color_discrete(h=0.1,s=0.8,l=0.5)
    + theme(
        figure_size=(7,5),
        legend_title=element_blank(),
    )
    + labs(
        x="Game # in Playoffs",
        title="Daily RAPM Estimate (DRE) for James Harden Playoff Runs",
        subtitle="DRE is a stat created by @NBACouchside similar to GameScore",
        caption="@sradjoker | source: nba.com/stats | DRE Formula: @NBACouchside",
    )
)
p

In [ ]:
dfa = []
for year in range(2023,2024):
    df1 = pd.read_parquet(box_DIR+f"NBA_Box_P_Base_{year}_PS.parquet")
    df1["GAME_ID"] = df1["GAME_ID"].astype(int)
    df2 = pd.read_parquet(box_DIR+f"NBA_Box_P_Adv_{year}_PS.parquet",columns=["gameId","personId","possessions"])
    df2.columns = ['GAME_ID','PLAYER_ID',"POSS"]
    df3 = pd.merge(df1,df2,on=['GAME_ID','PLAYER_ID'])
    df3["Season"] = year +1
    df3["DRE"] = -8.42364*df3["POSS"]/100+0.79231*df3["PTS"]-0.71944*(df3["FGA"]-df3["FG3A"])-0.55233*df3["FG3A"]-0.15944*df3["FTA"]+0.13479*df3["OREB"]+0.3996*df3["DREB"]+0.54415*df3["AST"]+1.68007*df3["STL"]+0.76387*df3["BLK"]-1.3599*df3["TOV"]-0.10838*df3["PF"]

    df3["GmSc"] = df3["PTS"]+0.4*df3["FGM"]-0.7*df3["FGA"]-0.4*(df3["FTA"]-df3["FTM"])+0.7*df3["OREB"]+0.3*df3["DREB"]+df3["STL"]+0.7*df3["AST"]+0.7*df3["BLK"]-0.4*df3["PF"]-df3["TOV"]
    dfa.append(df3)
df3 = pd.concat(dfa)

In [ ]:
players = ["LeBron James","Stephen Curry","Kevin Durant","Kawhi Leonard","DeMar DeRozan","Giannis Antetokounmpo","Luka Doncic","Shai Gilgeous-Alexander","Nikola Jokic","Jayson Tatum"]
# players = ["James Harden"]
# players = ["Joel Embiid"]
pIDs = [get_pID(pp) for pp in players]
df4 = df3[df3["PLAYER_ID"].isin(pIDs)]

In [ ]:
df6 = pd.DataFrame(df3.groupby(["PLAYER_ID","PLAYER_NAME"])["DRE"].mean()).reset_index().sort_values("DRE",ascending=False).reset_index(drop=True).reset_index()
df6["index"]+=1
df6["PLAYER_ID"] = df6["PLAYER_ID"].astype(str)
df6["DRE"] = round(df6["DRE"],1)
df6 = df6.tail(20)

In [ ]:
df6

In [ ]:
t = (
    GT(df6)
    .tab_header(
        title=md("Best Playoff Performers: using DRE"),
        subtitle="DRE: Daily RAPM estimate by @NBACouchside"
    )
    .tab_source_note(source_note="@sradjoker | source: nba.com/stats via nba_api")
    .cols_label(
        index = "#",
        PLAYER_ID = "",
        PLAYER_NAME = "Player",
    )
    .fmt_image(
        columns="PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="PLAYER_NAME")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
)
t.save(fig_DIR + "DRE_Playoffs_2024.png",scale=3,expand=2,web_driver="edge")
t